In [8]:
!which python

"which" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models.video import s3d

ModuleNotFoundError: No module named 'torch'

In [2]:
class VideoDataset(Dataset):
    def __init__(self, video_paths, labels, transform=None):
        """
        video_paths: список путей к видеофайлам
        labels: метки для каждого видео
        transform: трансформации для видео
        """
        self.video_paths = video_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.video_paths)

    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        
        # Загружаем видео (для примера используем OpenCV или PIL)
        video_frames = load_video_frames(video_path)  # Эта функция должна быть реализована
        
        if self.transform:
            video_frames = self.transform(video_frames)
        
        return video_frames, label

In [3]:
transform = transforms.Compose([
    transforms.Resize([256, 256]),
    transforms.CenterCrop([224, 224]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
])


In [5]:
model = s3d(weights='KINETICS400_V1')


In [10]:
labels = range(1000)

In [11]:
num_classes = len(set(labels))  # Количество классов в вашем датасете
model.fc = nn.Linear(model.fc.in_features, num_classes)


AttributeError: 'S3D' object has no attribute 'fc'

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    for batch_idx, (videos, labels) in enumerate(train_loader):
        videos, labels = videos.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Прогон модели
        outputs = model(videos)
        
        # Вычисление потерь
        loss = criterion(outputs, labels)
        
        # Обратное распространение ошибки и оптимизация
        loss.backward()
        optimizer.step()
        
        if batch_idx % 10 == 0:  # выводим информацию о процессе обучения
            print(f'Batch [{batch_idx}/{len(train_loader)}], Loss: {loss.item()}')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:
train_dataset = VideoDataset(video_paths=train_video_paths, labels=train_labels, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)


In [ ]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train(model, train_loader, criterion, optimizer, device)